# torch.autograd 를 사용한 자동 미분
---

이 부분은 우리가 Dezero를 만들면서 사용했던 이론 그대로 따라간다.

Dezero에선 variable이라는 클래스를 만들고 Function이라는 클래스를 만들었다

이때 Function이라는 클래스가 입력값(variable)과 출력값(variable)을 기억하고 저장함으로써 계산 그래프가 실시간으로 연결되어 갔다. 이 부분이 Define-and-run의 중요 부분이었다

마지막 출력값(variable)에서 역전파를 구하는 메서드인 backward()를 실행시켜주면 연결된 계산 그래프를 따라가서 미분값을 도출 할 수 있었다.

참고로 고계 미분의 경우 미분값을 도출할때 그려논 계산그래프를 다시 역전파 시켜버리는 엄청난 아이디어로 쉽게 구할 수 있었다!!! (아직도 이 아이디어에 대해서 충격을 받고 여운이 가시지 않는다)


### 1. Autograd 사용 방법
---
>참고 및 출처 - https://gaussian37.github.io/dl-pytorch-gradient/

- 어떤 temsor가 학습에 필요한 tensor라면 역전파를 통하여 기울기를 구해야 한다.

- tensor의 gradient를 구할 때에는 다음 조건들이 만족되어야 gradient를 구할 수 있습니다.

    - tensor의 옵션이 requires_grad = True 로 설정되어 있어야 합니다. (tensor의 기본 값은 requires_grad = False 입니다.)

    - backparopagation을 시작할 지점의 output은 scalar 형태이어야 합니다.

- tensor의 gradient를 구하는 방법은 backpropagation을 시작할 지점의 tensor에서 .backward() 함수를 호출하면 됩니다.

- gradient 값을 확인 하려면 requires_grad = True로 생성한 Tensor에서 .grad를 통해 값을 확인할 수 있습니다.

작동 원리가 Dezero에서 했던 방법이랑은 차이가 없다.


### 2. Dezero의 메모리 절약 방식
---
Dezero의 경우 혹시 어떻게 메모리를 절약했는지 기억이 나는가?

1. 중간 변수의 미분값 삭제

    첫번째 방법은 중간 변수의 미분값을 지워버리는 방식으로 해결했다.

    보통 우리는 말단 변수에서의 역전파를 구하고 싶어하지 중간 변수의 미분값을 안쓰는 경우가 많다. 그래서 중간 변수의 미분값을 아예 지워버렸다.

    Variable에 메서드 def backward(self, retain_grad = False), retain_grad = False 이라는 문장을 추가 함으로써 중간 미분 결과를 삭제했다. 

2. 계산 그래프의 연결을 하지 않음

    두번째 방법은 계산의 연결을 차단하는 방법이다. 

    계산 그래프는 순전파때 연결이 되어 역전파때 연결된 계산 그래프를 바탕으로 기울기를 구하는 방식이었다. 

    계산 그래프를 연결함과 동시에 입력값 또한 저장해야 했는데, 입력값이 역전파때 필요했기 때문이다. 

    보통 학습시에는 미분값을 구해야 하지만 추론 시에는 단순히 순전파만 보내기 떄문에 굳이 계산 그래프를 연결할 이유도 입력값을 저장할 이유도 없다.

    따라서 

    >class Config:  
    >  enable_backprop = True # True면 역전파 코드 활성화 False면 역전파 코드 비활성화

    라는 클래스를 추가해서 역전파 활성 모드와 역전파 비활성 모드를 전환했다. 


### 3. requires_grad=True
---
pytorch는 이러한 메모리 절약 방식에 requires_grad=True 라는 기능을 추가했는데

텐서에 requires_grad=True 라고 설정을 해주지 않으면(requires_grad = False, 디폴트 값이 False이다.) 텐서는 일반적인 텐서 연산만을 하게 된다 

In [ ]:
import torch

a = torch.tensor(2.0)
b = torch.tensor(3.0)
c = a * b # 그냥 일반적인 연산만 할 뿐이다

하지만 만일 requires_grad=True 라고 설정을 해줄 경우 이에 대한 모든 연산들을 추적하기 시작한다.

다시 말해 **계산 그래프를 연결하기 시작한다** 또한, 텐서에 역전파시 나온 기울기 값을 저장한다. 

즉 Dezero에서 메모리 절약 방식인 1번 방법과 2번방법을 합쳐서 한 문장에다가 넣은 것(!!!!!)

~~아 이거 찾아보고 생각하느라 개 빡셌네~~


In [ ]:
import torch

a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0)
c = a * b 
c.backward()

하지만 두 가지를 합쳤다고? 라는 생각에 이런 의문이 들 수 있다

그럼 위의 예시에서 a는 계산 그래프를 연결하고 추적하는데 b는 뭐 어떻게 되는건가요??

정답을 말하자면 b는 그냥 계산 그래프 연결에만 참여하고 기울기는 저장하지 않게 된다.

아래 예시에서 확인 할 수 있다.

> 예시 출처 : https://gaussian37.github.io/dl-pytorch-gradient/

In [ ]:
x = torch.randn(2, 2, requires_grad=True)
y = x + 2
z = (y * y).sum()
z.backward()

print(x)
# tensor([[ 2.5455,  1.3913],
#         [-0.4362, -0.5303]], requires_grad=True)

print(y)
# tensor([[4.5455, 3.3913],
#         [1.5638, 1.4697]], grad_fn=<AddBackward0>)

print(z)
# tensor(36.7684, grad_fn=<SumBackward0>)

print(x.grad)
# tensor([[9.0910, 6.7827],
#         [3.1276, 2.9394]])

print(y.grad)
# None

print(z.grad)
# None

>위 예제를 살펴보면 실제 값이 할당된 텐서인 x의 경우 backward를 통해 계산된 grad가 저장된 것을 볼 수 있습니다.  
>반면 계산 중간 과정인 y, z의 경우 실제 grad가 저장되진 않고 backward 연산에만 참여된 것으로 볼 수 있습니다.   
>그 결과 y.grad, z.grad는 None이 됩니다.  

또한 이러한 기능을 추가하면

각 텐서마다 계산 그래프를 형성하는 대단한 기능이 완성이 되기도 한다(!!!)

개개인 별로 각자의 계산 그래프를 만들어버릴 수 있단 뜻


### 4. no_gard()
---
> 기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다.  
> 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우에는, 이러한 추적이나 지원이 필요 없을 수 있습니다.  
> 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있습니다  



In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

### 5. backparopagation을 시작할 지점의 output은 scalar 형태이어야 합니다??? 
---
엥? 반드시 scalar 값이어야 한다고? Dezero에선 안그랬는데....?

그렇다

Dezero의 경우 출력값 또한 Variable로 출력되게 해서 딱히 행렬의 역전파 또한 문제가 없었다

Pytorch의 경우 문제가 생긴다 ~~왠진 모름~~

뭐 당연히 할 순 있다. tensor.backward()의 인자로 역전파를 하고 싶은 출력값을 넣어주면 된다. 

> 그래프는 연쇄 법칙을 사용하여 미분됩니다. 스칼라가 아닌 tensors 데이터가 있고(즉, 데이터에 두 개 이상의 요소가 있는 경우) 기울기가 필요한 경우 야코비안 벡터 곱이 계산됩니다. 


In [11]:
import torch

x = torch.randn(2, 2, requires_grad=True)
y = x + 2
z = (y * y)
y.backward(z)

print(x.grad)
# tensor([[2.2796, 3.2123],
#         [5.1224, 0.6321]])

tensor([[2.4366, 0.5017],
        [0.5960, 7.6071]])
